In [1]:
import librosa
import soundfile as sf
import numpy as np

In [2]:
# Хочется придумать код который бы сохранил один .csv
# Внутри его будет 
# path to mixed_audio | path to audio_1 | path to audio_2

For a sample

In [3]:
# Загрузка двух WAV файлов
y1, sr1 = librosa.load("C:/Users/b.smadiarov/Diploma/DualPathRNN/data/samples/5ed8a1c0f3ea2.flac", sr=16000)
y2, sr2 = librosa.load("C:/Users/b.smadiarov/Diploma/DualPathRNN/data/samples/5f2b0a06ceb18.flac", sr=16000)

In [6]:
# Приведение длительности и частоты дискретизации к одинаковым значениям
if sr1 != sr2:
    raise ValueError("Частоты дискретизации файлов должны совпадать!")
min_length = min(len(y1), len(y2))
y1 = y1[:min_length]
y2 = y2[:min_length]

# Наложение (суммирование сигналов)
overlay = y1 + y2
# overlay = np.clip(overlay, -1.0, 1.0)  # Для предотвращения переполнения

sf.write("C:/Users/b.smadiarov/Diploma/DualPathRNN/data/samples/5ed8a1c0f3ea2_5f2b0a06ceb18.flac", overlay, sr1)

For samples